<a href="https://colab.research.google.com/github/rohit112121/Healthcare-fraud/blob/main/healthcare_fraud__Autoencoder_and_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'healthcare-provider-fraud-detection-analysis:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F188596%2F421248%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240904%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240904T201637Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2ca5d9386f568dd807297347059007087a675db97a908e78f39db497820cf7bfbb8a9cc68c38d3783e94e4f6805d28987656b00e2f7ca81aaf7d3071ae0b2ee15dfb1f56bd5273d4f81987af518fbb693f1c5454931908ab75ca0017547a149b400289ea04b6f034155813d6b1cf98ee91e868cf2ade16de3794009d234bc6391a5a85bfcdcdcf54bcaca4b6ee47772bcc2c05f69cca8b15cbc54194824c1594b79bcd68dd3eab010a8956109bb61fa19ae6749a5d58c28afa87df3b5141b1ffe594a1adaedef19bca6ee9fe371dad8117c512569196fbba56cffeb482064f9b070ec86946aa33f59b64f3d9eb656b9e68139f1d433e6b05321ea2fc386b5b14'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 26631783 bytes downloaded
Downloaded and uncompressed: healthcare-provider-fraud-detection-analysis
Data source import complete.


In [ ]:
!pip install imbalanced-learn

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder , LabelEncoder
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
Train=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train-1542865627584.csv")
Train_Beneficiarydata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Beneficiarydata-1542865627584.csv")
Train_Inpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Inpatientdata-1542865627584.csv")
Train_Outpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Outpatientdata-1542865627584.csv")

# Merging data

In [ ]:
Train_Allpatientdata=pd.merge(Train_Outpatientdata,Train_Inpatientdata,
                              left_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
                              right_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode']
                              ,how='outer')

In [ ]:
Train_Allpatientdata=pd.merge(Train_Allpatientdata,Train_Beneficiarydata,on="BeneID")

In [ ]:
Train_Allpatientdata=pd.merge(Train_Allpatientdata,Train,on="Provider")

In [ ]:
Train_Allpatientdata.shape

(558211, 55)

In [ ]:
Train_Allpatientdata.head(5)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,2,2,2,2,2,0,0,30,50,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,1,1,1,1,2,0,0,1810,760,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,1,1,1,1,2,0,0,1810,760,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,1,1,1,1,2,0,0,1810,760,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,1,1,1,1,2,0,0,1810,760,Yes


In [ ]:
Train_Allpatientdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 55 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   ClaimID                          558211 non-null  object 
 2   ClaimStartDt                     558211 non-null  object 
 3   ClaimEndDt                       558211 non-null  object 
 4   Provider                         558211 non-null  object 
 5   InscClaimAmtReimbursed           558211 non-null  int64  
 6   AttendingPhysician               556703 non-null  object 
 7   OperatingPhysician               114447 non-null  object 
 8   OtherPhysician                   199736 non-null  object 
 9   ClmDiagnosisCode_1               547758 non-null  object 
 10  ClmDiagnosisCode_2               362605 non-null  object 
 11  ClmDiagnosisCode_3               243055 non-null  object 
 12  Cl

# Physician cols preprocessing

In [ ]:
a=(Train_Allpatientdata["AttendingPhysician"]==Train_Allpatientdata["OperatingPhysician"])
b=(Train_Allpatientdata["OperatingPhysician"]==Train_Allpatientdata["OtherPhysician"])
c=(Train_Allpatientdata["AttendingPhysician"]==Train_Allpatientdata["OtherPhysician"])

print(a.sum())
print(b.sum())
print(c.sum())
print( (a+b).sum() ) # atten=oper=other

61164
15884
85258
63872


In [ ]:
def physician_same(row):
    atten_oper=row["AttendingPhysician"]==row["OperatingPhysician"]
    oper_other=row["OperatingPhysician"]==row["OtherPhysician"]
    atten_other=row["AttendingPhysician"]==row["OtherPhysician"]
    if atten_oper==True and oper_other==True:# atten = oper = other
        return 0
    elif atten_oper==True and oper_other==False:# atten = oper != other
        return 1
    elif atten_oper==False and oper_other==True:# atten != oper = other
        return 2
    else:# atten != oper != other
        return 3

phy_same=Train_Allpatientdata.apply(physician_same,axis=1)

In [ ]:
Train_Allpatientdata["phy_same"]=phy_same

In [ ]:
def physician_count(row,list_count):
    count=0
    for col in list_count:
        if pd.isnull(row[col]):
            continue
        else:
            count+=1
    return count

list_count=["AttendingPhysician","OperatingPhysician","OtherPhysician"]
phy_count=Train_Allpatientdata.apply(physician_count,axis=1,args=(list_count,))

In [ ]:
Train_Allpatientdata["phy_count"]=phy_count

In [ ]:
Train_Allpatientdata.head(4)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,phy_same,phy_count
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,2,2,2,0,0,30,50,Yes,3,1
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,1,1,2,0,0,1810,760,Yes,3,1
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,1,1,2,0,0,1810,760,Yes,3,1
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,1,1,2,0,0,1810,760,Yes,3,1


# ClainStartDt , ClainEndDt cols preprocessing

In [ ]:
startdate= pd.to_datetime( Train_Allpatientdata["ClaimStartDt"] )
enddate= pd.to_datetime( Train_Allpatientdata["ClaimEndDt"] )

period = ( enddate - startdate).dt.days
Train_Allpatientdata["period"] = period

In [ ]:
Train_Allpatientdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 58 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   ClaimID                          558211 non-null  object 
 2   ClaimStartDt                     558211 non-null  object 
 3   ClaimEndDt                       558211 non-null  object 
 4   Provider                         558211 non-null  object 
 5   InscClaimAmtReimbursed           558211 non-null  int64  
 6   AttendingPhysician               556703 non-null  object 
 7   OperatingPhysician               114447 non-null  object 
 8   OtherPhysician                   199736 non-null  object 
 9   ClmDiagnosisCode_1               547758 non-null  object 
 10  ClmDiagnosisCode_2               362605 non-null  object 
 11  ClmDiagnosisCode_3               243055 non-null  object 
 12  Cl

## taking copy

In [ ]:
copy1=Train_Allpatientdata.copy()

# Cronic code cols preprocessing

In [ ]:
cronic_cols_names=copy1.columns[ copy1.columns.str.startswith("ChronicCond") ]
cronic_cols=copy1[   cronic_cols_names   ]
cronic=cronic_cols.replace({2:0})
copy1[   cronic_cols_names   ]=cronic

# Potintial fraud preprocessing

In [ ]:
copy1["PotentialFraud"]=copy1["PotentialFraud"].replace({"Yes":1,"No":0})

# gender preprocessing

In [ ]:
copy1["Gender"]=copy1["Gender"].replace({2:0})

# Admisson data preprocessing

In [ ]:
startadmt= pd.to_datetime( copy1["AdmissionDt"] )
enddatadmt= pd.to_datetime( copy1["DischargeDt"] )

periodadmt = ( enddatadmt - startadmt).dt.days
copy1["periodadmt"] = periodadmt
copy1["periodadmt"]=copy1["periodadmt"].fillna(0)

# RenalDiseaseIndicator preprocessing

In [ ]:
copy1["RenalDiseaseIndicator"]=copy1["RenalDiseaseIndicator"].replace({"Y":1})

# patient age preprocessing

In [ ]:
birthdate=pd.to_datetime(copy1["DOB"])
enddate=pd.to_datetime(copy1["DOD"])

# cheack whether the patient dead or alife
def alife_function(value):
    if value==True:
        return 1
    else:
        return 0
alife = pd.isna(enddate).apply(alife_function)


# get the age of patient
max_date=enddate.dropna().max()
enddate[pd.isna(enddate)]=max_date
period=(((enddate-birthdate).dt.days/356).astype(int))

copy1["age"]=period
copy1["alife"]=alife

In [ ]:
copy1.head(2)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,phy_same,phy_count,period,periodadmt,age,alife
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,0,30,50,1,3,1,0,0.0,75,1
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,0,1810,760,1,3,1,0,0.0,89,1


In [ ]:
copy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 61 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   ClaimID                          558211 non-null  object 
 2   ClaimStartDt                     558211 non-null  object 
 3   ClaimEndDt                       558211 non-null  object 
 4   Provider                         558211 non-null  object 
 5   InscClaimAmtReimbursed           558211 non-null  int64  
 6   AttendingPhysician               556703 non-null  object 
 7   OperatingPhysician               114447 non-null  object 
 8   OtherPhysician                   199736 non-null  object 
 9   ClmDiagnosisCode_1               547758 non-null  object 
 10  ClmDiagnosisCode_2               362605 non-null  object 
 11  ClmDiagnosisCode_3               243055 non-null  object 
 12  Cl

# means and stds

In [ ]:
def groupby(df,by,vars_to_group,methods,col_ident,as_index=True,agg=False):
    if agg:
        grouped=df.groupby(by=by,as_index=as_index)[vars_to_group].agg(methods)
        cols=['_'.join(col) for col in grouped.columns.values]
        cols=[col_ident+"_"+col for col in cols]
        grouped.columns=cols
        return grouped

    else:
        concat=df.groupby(by=by,as_index=as_index)[vars_to_group].transform(methods[0])
        cols=[ col_ident+"_"+col+"_"+methods[0] for col in concat.columns ]
        concat.columns=cols

        for method in methods[1:]:
            grouped=df.groupby(by=by,as_index=as_index)[vars_to_group].transform(method)
            cols=[col_ident+"_"+col+"_"+method for col in grouped.columns]
            grouped.columns=cols
            concat=pd.concat([concat,grouped],axis=1)

        return concat

In [ ]:
money_cols=["InscClaimAmtReimbursed","DeductibleAmtPaid","NoOfMonths_PartACov","NoOfMonths_PartBCov",
           "IPAnnualReimbursementAmt","IPAnnualDeductibleAmt","OPAnnualReimbursementAmt","OPAnnualDeductibleAmt"]

In [ ]:
provider_money=groupby(copy1,["Provider"],money_cols,["mean","std"],"provider",
                       True,False)

In [ ]:
banel_money=groupby(copy1,["BeneID"],money_cols,["mean","std"],"banel",
                       True,False)

In [ ]:
diag1_money=groupby(copy1,["ClmDiagnosisCode_1"],money_cols,["mean","std"],"diag1",
                       True,False)

In [ ]:
selected_cols_names=["phy_same","phy_count","period","periodadmt","age","alife","Provider","PotentialFraud"]
selected_cols=copy1[selected_cols_names]

In [ ]:
data=pd.concat([selected_cols,provider_money,banel_money,diag1_money],axis=1)

In [ ]:
grouped=data.groupby(by=["Provider","PotentialFraud"]).agg("mean").reset_index()

In [ ]:
grouped

,Provider,PotentialFraud,phy_same,phy_count,period,periodadmt,age,alife,provider_InscClaimAmtReimbursed_mean,provider_DeductibleAmtPaid_mean,...,diag1_OPAnnualReimbursementAmt_mean,diag1_OPAnnualDeductibleAmt_mean,diag1_InscClaimAmtReimbursed_std,diag1_DeductibleAmtPaid_std,diag1_NoOfMonths_PartACov_std,diag1_NoOfMonths_PartBCov_std,diag1_IPAnnualReimbursementAmt_std,diag1_IPAnnualDeductibleAmt_std,diag1_OPAnnualReimbursementAmt_std,diag1_OPAnnualDeductibleAmt_std
0,PRV51001,0,2.960000,1.600000,1.440000,1.000000,80.240000,1.000000,4185.600000,213.600000,...,1804.331094,543.045084,3482.066310,161.353027,0.569945,0.424192,12941.552352,1205.297144,2450.076771,661.506672
1,PRV51003,1,2.439394,1.530303,3.674242,2.424242,71.371212,0.992424,4588.409091,502.166667,...,2422.458599,676.313985,4017.871066,260.257069,0.726572,0.653285,12620.604411,1226.306633,3369.338617,848.213675
2,PRV51004,0,2.818792,1.604027,1.429530,0.000000,73.516779,0.993289,350.134228,2.080537,...,2430.017927,694.246881,1536.290845,113.086257,0.667719,0.577420,11016.516937,1111.592405,2972.377916,808.138208
3,PRV51005,1,2.731330,1.599142,1.088412,0.000000,71.783691,0.996567,241.124464,3.175966,...,2195.953526,630.805985,1234.005090,91.141252,0.657071,0.565930,10021.329572,957.701391,2727.944083,737.419878
4,PRV51007,0,2.736111,1.527778,0.958333,0.222222,70.583333,0.986111,468.194444,45.333333,...,2089.969651,606.550334,1519.425993,103.302166,0.626542,0.520122,10565.761429,1126.358206,2486.827069,682.279276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0,2.892857,1.464286,2.142857,0.000000,75.285714,1.000000,380.000000,4.642857,...,2770.995487,762.684094,924.661538,88.673186,0.717266,0.575517,11706.835391,989.925167,3092.516228,805.101036
5406,PRV57760,0,2.318182,1.681818,0.318182,0.000000,62.318182,1.000000,216.818182,0.000000,...,1857.020747,551.812490,1182.800362,63.746284,0.738778,0.632455,10087.266945,852.726747,2325.052621,667.813928
5407,PRV57761,0,3.000000,1.609756,1.390244,0.000000,73.268293,0.987805,225.243902,4.512195,...,2409.246660,667.100333,1469.587798,89.369298,0.626573,0.503166,9701.587916,956.791480,2864.403847,730.914731
5408,PRV57762,0,3.000000,1.000000,0.000000,0.000000,69.000000,1.000000,1900.000000,0.000000,...,2332.103825,701.530055,926.436661,146.810805,0.939796,0.000000,10836.121659,798.136760,2327.892739,625.238022


In [ ]:
grouped=grouped.fillna(0)

In [ ]:
features=grouped.iloc[:,2:]
labels=grouped.iloc[:,1]

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
features, labels = oversample.fit_resample(features, labels)

In [ ]:
scaler = StandardScaler()
featuresstand=scaler.fit_transform(features)

In [ ]:
ff=compute_class_weight(class_weight="balanced",classes=np.unique(labels),y=labels)
cw=dict(zip(np.unique(labels),ff))

featuress,labelss=shuffle(featuresstand,labels)
xtrain,xtest,ytrain,ytest = train_test_split(featuress,labelss,test_size=0.2)

In [ ]:

xtrain=xtrain.astype(np.float32)
xtest=xtest.astype(np.float32)
ytrain=ytrain.astype(np.float32).to_numpy()
ytest=ytest.astype(np.float32).to_numpy()


ytrain=ytrain.reshape(ytrain.shape+(1,))
ytest=ytest.reshape(ytest.shape+(1,))

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(7846, 54)
(1962, 54)
(7846, 1)
(1962, 1)


In [ ]:
inpt=tf.keras.layers.Input((xtrain.shape[1],))
d1=tf.keras.layers.Dense(256, activation='relu')(inpt)
d1=tf.keras.layers.Dense(128, activation='relu')(d1)

d2=tf.keras.layers.Dense(1,activation="sigmoid")(d1)

nural_network=tf.keras.Model(inputs=inpt,outputs=d2)

In [ ]:
nural_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss='binary_crossentropy'
                    ,metrics=["accuracy"])

history_combined=nural_network.fit(xtrain,ytrain,validation_data=(xtest,ytest),batch_size=128,epochs=300,
                                  class_weight=cw)

Epoch 1/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6518 - loss: 0.6088 - val_accuracy: 0.7370 - val_loss: 0.5024
Epoch 2/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7363 - loss: 0.4946 - val_accuracy: 0.8104 - val_loss: 0.4385
Epoch 3/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8130 - loss: 0.4293 - val_accuracy: 0.8445 - val_loss: 0.3958
Epoch 4/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8509 - loss: 0.3856 - val_accuracy: 0.8573 - val_loss: 0.3655
Epoch 5/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8565 - loss: 0.3563 - val_accuracy: 0.8634 - val_loss: 0.3404
Epoch 6/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8627 - loss: 0.3393 - val_accuracy: 0.8710 - val_loss: 0.3249
Epoch 7/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8639 - loss: 0.3255 - val_accuracy: 0.8741 - val_loss: 0.3139
Epoch 8/300
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8773 - loss: 0.3065 - val_accuracy: 0.8782 - v

In [ ]:
ytt=nural_network.predict(xtest)
dd=np.round(ytt)
print(classification_report(ytest,dd))
_, auc_score = nural_network.evaluate(xtest, ytest)
print("AUROC Score:", auc_score)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97      1022
         1.0       0.95      0.99      0.97       940

    accuracy                           0.97      1962
   macro avg       0.97      0.97      0.97      1962
weighted avg       0.97      0.97      0.97      1962

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9654 - loss: 0.1447
AUROC Score: 0.9689092636108398


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# ... (your existing code for data preprocessing)

# Define the autoencoder architecture
input_dim = xtrain.shape[1]
encoding_dim = 32  # Adjust encoding dimension as needed

input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(encoder)  # Bottleneck layer

decoder = tf.keras.layers.Dense(64, activation='relu')(encoded)
decoder = tf.keras.layers.Dense(128, activation='relu')(decoder)
decoded = tf.keras.layers.Dense(input_dim, activation='linear')(decoder)  # Reconstruction layer

autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse', metrics=["accuracy"], metrics=[tf.keras.metrics.AUC(name='auc')])  # Mean squared error for reconstruction loss

# Train the autoencoder
autoencoder.fit(xtrain, xtrain, epochs=200, batch_size=32, validation_data=(xtest, xtest))  # Adjust epochs and batch size

# Extract the encoded representations (latent features)
encoder_model = tf.keras.Model(inputs=input_layer, outputs=encoded)
encoded_xtrain = encoder_model.predict(xtrain)
encoded_xtest = encoder_model.predict(xtest)

# Now you can use encoded_xtrain and encoded_xtest as features for your classification task
# ... (continue with your classification model using encoded features)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Define the autoencoder architecture
input_dim = xtrain.shape[1]
encoding_dim = 32  # Adjust encoding dimension as needed

input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(encoder)  # Bottleneck layer

decoder = tf.keras.layers.Dense(64, activation='relu')(encoded)
decoder = tf.keras.layers.Dense(128, activation='relu')(decoder)
decoded = tf.keras.layers.Dense(input_dim, activation='linear')(decoder)  # Reconstruction layer

autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse', metrics=["accuracy"])  # Mean squared error for reconstruction loss

# Train the autoencoder
autoencoder.fit(xtrain, xtrain, epochs=100, batch_size=32, validation_data=(xtest, xtest))  # Adjust epochs and batch size

# Extract the encoded representations (latent features)
encoder_model = tf.keras.Model(inputs=input_layer, outputs=encoded)
encoded_xtrain = encoder_model.predict(xtrain)
encoded_xtest = encoder_model.predict(xtest)

# Define the classification model using encoded features
inpt = tf.keras.layers.Input(shape=(encoded_xtrain.shape[1],))
d1 = tf.keras.layers.Dense(128, activation='relu')(inpt)
d1 = tf.keras.layers.Dense(64, activation='relu')(d1)
d2 = tf.keras.layers.Dense(1, activation="sigmoid")(d1)
classifier = tf.keras.Model(inputs=inpt, outputs=d2)

# Compile the classifier
classifier.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=[tf.keras.metrics.AUC(name='auc')])

# Train the classifier
classifier.fit(encoded_xtrain, ytrain, epochs=100, batch_size=32, validation_data=(encoded_xtest, ytest))

# Predict on test data
y_pred_proba = classifier.predict(encoded_xtest)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

Epoch 1/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.1095 - loss: 0.4873 - val_accuracy: 0.3496 - val_loss: 0.2326
Epoch 2/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3959 - loss: 0.2230 - val_accuracy: 0.4679 - val_loss: 0.1653
Epoch 3/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4679 - loss: 0.1841 - val_accuracy: 0.5015 - val_loss: 0.1320
Epoch 4/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5322 - loss: 0.1151 - val_accuracy: 0.4766 - val_loss: 0.1345
Epoch 5/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5512 - loss: 0.1147 - val_accuracy: 0.5846 - val_loss: 0.0918
Epoch 6/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6081 - loss: 0.0830 - val_accuracy: 0.5841 - val_loss: 0.0965
Epoch 7/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6271 - loss: 0.0809 - val_accuracy: 0.6162 - val_loss: 0.0813
Epoch 8/100
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6535 - loss: 0.0714 - val_accu

In [ ]:
#LOGISTIC REGRESSION


import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Initialize Logistic Regression model with increased max_iter
logistic_model = LogisticRegression(max_iter=1000)  # Increase as needed

# Train the model
logistic_model.fit(xtrain, ytrain.ravel())  # Flatten ytrain if it's a column vector

# Predict on test data
y_pred = logistic_model.predict(xtest)
y_pred_proba = logistic_model.predict_proba(xtest)[:, 1]  # Probabilities for positive class

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

              precision    recall  f1-score   support

         0.0       0.87      0.80      0.83      1022
         1.0       0.80      0.87      0.84       940

    accuracy                           0.83      1962
   macro avg       0.84      0.84      0.83      1962
weighted avg       0.84      0.83      0.83      1962

AUROC Score: 0.9054856976308446


In [ ]:
#RANDOM FOREST

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust n_estimators as needed

# Train the model
rf_model.fit(xtrain, ytrain.ravel())  # Flatten ytrain if it's a column vector

# Predict on test data
y_pred = rf_model.predict(xtest)
y_pred_proba = rf_model.predict_proba(xtest)[:, 1]  # Probabilities for positive class

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95      1022
         1.0       0.94      0.97      0.95       940

    accuracy                           0.95      1962
   macro avg       0.95      0.95      0.95      1962
weighted avg       0.95      0.95      0.95      1962

AUROC Score: 0.9919104176208519


In [ ]:
#XGBoost

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Initialize XGBoost model (without use_label_encoder)
xgb_model = XGBClassifier(eval_metric='logloss')  # Adjust other parameters as needed

# Train the model
xgb_model.fit(xtrain, ytrain.ravel())  # Flatten ytrain if it's a column vector

# Predict on test data
y_pred = xgb_model.predict(xtest)
y_pred_proba = xgb_model.predict_proba(xtest)[:, 1]  # Probabilities for positive class

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

              precision    recall  f1-score   support

         0.0       0.98      0.95      0.96      1022
         1.0       0.94      0.98      0.96       940

    accuracy                           0.96      1962
   macro avg       0.96      0.96      0.96      1962
weighted avg       0.96      0.96      0.96      1962

AUROC Score: 0.9945496939667735


In [ ]:
!pip install lightgbm

In [ ]:
#LightGBM

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Initialize LightGBM model with force_col_wise
lgb_model = LGBMClassifier(force_col_wise=True)  # Adjust other parameters as needed

# Train the model
lgb_model.fit(xtrain, ytrain.ravel())  # Flatten ytrain if it's a column vector

# Predict on test data
y_pred = lgb_model.predict(xtest)
y_pred_proba = lgb_model.predict_proba(xtest)[:, 1]  # Probabilities for positive class

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

[LightGBM] [Info] Number of positive: 3964, number of negative: 3882
[LightGBM] [Info] Total Bins 12239
[LightGBM] [Info] Number of data points in the train set: 7846, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505226 -> initscore=0.020903
[LightGBM] [Info] Start training from score 0.020903
              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96      1022
         1.0       0.94      0.98      0.96       940

    accuracy                           0.96      1962
   macro avg       0.96      0.96      0.96      1962
weighted avg       0.96      0.96      0.96      1962

AUROC Score: 0.9927801140858559


In [ ]:
#MLP

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# ... (your existing code for data preprocessing)

# Initialize Multi-Layer Perceptron model
mlp_model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=200, random_state=42)
# Adjust hidden_layer_sizes, activation, solver, max_iter, and random_state as needed

# Train the model
mlp_model.fit(xtrain, ytrain.ravel())  # Flatten ytrain if it's a column vector

# Predict on test data
y_pred = mlp_model.predict(xtest)
y_pred_proba = mlp_model.predict_proba(xtest)[:, 1]  # Probabilities for positive class

# Print classification report
print(classification_report(ytest, y_pred))

# Calculate and print AUROC score
auc_score = roc_auc_score(ytest, y_pred_proba)
print("AUROC Score:", auc_score)

              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97      1022
         1.0       0.95      0.99      0.97       940

    accuracy                           0.97      1962
   macro avg       0.97      0.97      0.97      1962
weighted avg       0.97      0.97      0.97      1962

AUROC Score: 0.9902100595411585
